In [1]:
text = "Hola como estas"

In [ ]:
import torch
import zipfile
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU
model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='es', # also available 'de', 'es'
                                       device=device)
read_batch, split_into_batches, read_audio, prepare_model_input = utils  # see function signature for details

In [11]:
test_files = glob('Subject_Jose_T=1.wav')
batches = split_into_batches(test_files, batch_size=1)
input = prepare_model_input(read_batch(batches[0]), device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))

sí que yo me llamo josé y estoy muy contento de conocer te maría


In [8]:
from google.cloud import speech_v1 as speech


def speech_to_text(config, audio):
    client = speech.SpeechClient.from_service_account_json("../google_api.json")
    response = client.recognize(config=config, audio=audio)
    print_sentences(response)


def print_sentences(response):
    for result in response.results:
        best_alternative = result.alternatives[0]
        transcript = best_alternative.transcript
        confidence = best_alternative.confidence
        print("-" * 80)
        print(f"Transcript: {transcript}")
        print(f"Confidence: {confidence:.0%}")


config = dict(language_code="en-US")
config.update(dict(enable_automatic_punctuation=True))
audio = dict(uri="gs://cloud-samples-data/speech/brooklyn_bridge.flac")


In [9]:
speech_to_text(config, audio)

--------------------------------------------------------------------------------
Transcript: How old is the Brooklyn Bridge?
Confidence: 98%


In [1]:
from google.cloud import speech

client = speech.SpeechClient.from_service_account_json("../google_api.json")

In [2]:
speech_file = "Subject_Jose_T=0.wav"
with open(speech_file, "rb") as audio_file:
    content = audio_file.read()

In [3]:
audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    # sample_rate_hertz=44100,
    language_code="es-EU",
    audio_channel_count=2,
    enable_separate_recognition_per_channel=True,
    enable_automatic_punctuation=True
)

response = client.recognize(config=config, audio=audio)

for i, result in enumerate(response.results):
    alternative = result.alternatives[0]
    print("-" * 20)
    print("First alternative of result {}".format(i))
    print(u"Transcript: {}".format(alternative.transcript))
    print(u"Channel Tag: {}".format(result.channel_tag))

--------------------
First alternative of result 0
Transcript: Hola, María cómo estás? Yo también estoy muy contenta de conocerte.
Channel Tag: 1
--------------------
First alternative of result 1
Transcript: Hola, María cómo estás? Yo también estoy muy contenta de conocerte.
Channel Tag: 2


In [12]:
response.results[0].alternatives[0].transcript

'Hola, María cómo estás? Yo también estoy muy contenta de conocerte.'